<a href="https://colab.research.google.com/github/matchbou/kaggle-study-20190410/blob/master/titanic-catboost_Kaggle.pdf-201904161330.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# https://www.kaggle.com/dlarionov/titanic-catboost

# Titanic catboost

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"matchbou","key":"fb2093338e9cd630d62e0949908398dc"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!pip install kaggle

In [0]:
!chmod 600 /root/.kaggle/kaggle.json

In [0]:
# kaggleデータ置き場とjupyernoteのカレントディレクトリフォルダを作成

!mkdir ./input
!mkdir ./notebook

In [7]:
#osライブラリ読込してinputに移動⇒kaggleデータもダウンロード　⇒ notebookフォルダに移動
import os
os.chdir('input')

#https://www.kaggle.com/c/titanic/data
!kaggle competitions download -c titanic

os.chdir('../notebook')


  0% 0.00/59.8k [00:00<?, ?B/s]
100% 59.8k/59.8k [00:00<00:00, 52.1MB/s]
  0% 0.00/28.0k [00:00<?, ?B/s]
100% 28.0k/28.0k [00:00<00:00, 27.4MB/s]
  0% 0.00/3.18k [00:00<?, ?B/s]
100% 3.18k/3.18k [00:00<00:00, 2.85MB/s]


In [8]:
#上記操作の結果以下のディレクトリ配置となる

# /content/notebook このipynbのカレントディレクトリ
# /content/input カグルからダウンロードしたcsvファイルの置き場所

!pwd
!ls -a ../input


/content/notebook
.  ..  gender_submission.csv  test.csv	train.csv


In [0]:
#ここからカーネルの内容に踏み込む

In [11]:
!pip install catboost

    100% |████████████████████████████████| 60.6MB 677kB/s 


In [0]:
import numpy as np
import pandas as pd

from catboost import CatBoostClassifier, Pool, cv

import hyperopt

import sys

In [15]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

train_size = train.shape[0] # 891
test_size = test.shape[0] # 418

data = pd.concat([train, test])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


In [0]:
data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=False)

In [0]:
age_ref = data.groupby('Title').Age.mean()
data['Age'] = data.apply(lambda r: r.Age if pd.notnull(r.Age) else age_ref[r.Title] , axis=1)
del age_ref

In [0]:
data.loc[(data.PassengerId==1044, 'Fare')] = 14.43

In [0]:
data['Embarked'] = data['Embarked'].fillna('S')
data['Cabin'] = data['Cabin'].fillna('Undefined')

In [21]:
cols = [
    'Pclass',
    'Name',
    'Sex',
    'Age',
    'SibSp',
    'Parch',
    'Ticket',
    'Fare',
    'Cabin',
    'Embarked'
]
X_train = data[:train_size][cols]
Y_train = data[:train_size]['Survived'].astype(int)
X_test = data[train_size:][cols]

categorical_features_indices = [0,1,2,6,8,9]
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Undefined,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Undefined,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Undefined,S


In [0]:
train_pool = Pool(X_train, Y_train, cat_features=categorical_features_indices)

In [23]:
'''
def hyperopt_objective(params):
    model = CatBoostClassifier(
        l2_leaf_reg=int(params['l2_leaf_reg']),
        learning_rate=params['learning_rate'],
        depth=params['depth'],
        iterations=500,
        eval_metric='Accuracy',
        od_type='Iter',
        od_wait=40,
        random_seed=42,
        logging_level='Silent',
        allow_writing_files=False
    )
    
    cv_data = cv(
        train_pool,
        model.get_params()
    )
    best_accuracy = np.max(cv_data['test-Accuracy-mean'])    
    
    print(params, best_accuracy)
    return 1 - best_accuracy # as hyperopt minimises

params_space = {
    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),
    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),
    'depth': hyperopt.hp.choice('depth', [3,4,5,6,8]),
}

trials = hyperopt.Trials()

best = hyperopt.fmin(
    hyperopt_objective,
    space=params_space,
    algo=hyperopt.tpe.suggest,
    max_evals=50,
    trials=trials
)

print(best)
'''

"\ndef hyperopt_objective(params):\n    model = CatBoostClassifier(\n        l2_leaf_reg=int(params['l2_leaf_reg']),\n        learning_rate=params['learning_rate'],\n        depth=params['depth'],\n        iterations=500,\n        eval_metric='Accuracy',\n        od_type='Iter',\n        od_wait=40,\n        random_seed=42,\n        logging_level='Silent',\n        allow_writing_files=False\n    )\n    \n    cv_data = cv(\n        train_pool,\n        model.get_params()\n    )\n    best_accuracy = np.max(cv_data['test-Accuracy-mean'])    \n    \n    print(params, best_accuracy)\n    return 1 - best_accuracy # as hyperopt minimises\n\nparams_space = {\n    'l2_leaf_reg': hyperopt.hp.qloguniform('l2_leaf_reg', 0, 2, 1),\n    'learning_rate': hyperopt.hp.uniform('learning_rate', 1e-3, 5e-1),\n    'depth': hyperopt.hp.choice('depth', [3,4,5,6,8]),\n}\n\ntrials = hyperopt.Trials()\n\nbest = hyperopt.fmin(\n    hyperopt_objective,\n    space=params_space,\n    algo=hyperopt.tpe.suggest,\n   

In [24]:
#best = {'depth': 6, 'l2_leaf_reg': 1.0, 'learning_rate': 0.07395682681736576} 

model = CatBoostClassifier(
    #l2_leaf_reg=int(best['l2_leaf_reg']),
    #learning_rate=best['learning_rate'],
    #depth=best['depth'],
    depth=3,
    iterations=300,
    eval_metric='Accuracy',
    #od_type='Iter',
    #od_wait=40,
    random_seed=42,
    logging_level='Silent',
    allow_writing_files=False
)

cv_data = cv(
    train_pool,
    model.get_params(),
    fold_count=5
)

print('Best validation accuracy score: {:.2f}±{:.2f} on step {}'.format(
    np.max(cv_data['test-Accuracy-mean']), 
    cv_data['test-Accuracy-std'][cv_data['test-Accuracy-mean'].idxmax(axis=0)],
    cv_data['test-Accuracy-mean'].idxmax(axis=0)
))
print('Precise validation accuracy score: {}'.format(np.max(cv_data['test-Accuracy-mean'])))

model.fit(train_pool);
model.score(X_train, Y_train)

Best validation accuracy score: 0.84±0.04 on step 122
Precise validation accuracy score: 0.8373109032703534


0.9113355780022446

In [25]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

Dataset is provided, but PredictionValuesChange feature importance don't use it, since non-empty LeafWeights in model.
Sex: 39.22154684816448
Ticket: 14.997796628086467
Age: 13.938514894512783
Pclass: 12.201693067422497
Cabin: 6.118824179858983
Fare: 5.546108572014286
Embarked: 3.6254778390095748
SibSp: 3.624456883823927
Parch: 0.7255810871069684
Name: 0.0


In [0]:
Y_pred = model.predict(X_test)

submission = pd.DataFrame({
    "PassengerId": data[train_size:]["PassengerId"], 
    "Survived": Y_pred.astype(int)
})
submission.to_csv('Titanic_catboost_submission.csv', index=False)

In [0]:
from google.colab import files


files.download('Titanic_catboost_submission.csv')